# **SAR Target/Object Detection Using Machine Learning**

The notebook represents real time implementation of data processing, model building and testing of multiple Machine Learning Algorithms for detection of the target in MSTAR dataset.

**Mounting Google Drive for Cloud based Processing using GPUs**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Necassary Packages Installation**

In [35]:
!pip install scipy==1.2.1

  Using cached scipy-1.2.1.tar.gz (23.1 MB)
  Running setup.py clean for scipy
Failed to build scipy
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.2
    Uninstalling scipy-1.5.2:
      Successfully uninstalled scipy-1.5.2
    Running setup.py install for scipy: started
    Running setup.py install for scipy: finished with status 'error'
  Rolling back uninstall of scipy
  Moving to d:\anaconda3\lib\site-packages\scipy-1.5.2.dist-info\


  ERROR: Command errored out with exit status 1:
   command: 'D:\Anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Eric\\AppData\\Local\\Temp\\pip-install-bpwo_rzc\\scipy\\setup.py'"'"'; __file__='"'"'C:\\Users\\Eric\\AppData\\Local\\Temp\\pip-install-bpwo_rzc\\scipy\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Eric\AppData\Local\Temp\pip-wheel-bjmwb8r6'
       cwd: C:\Users\Eric\AppData\Local\Temp\pip-install-bpwo_rzc\scipy\
  Complete output (122 lines):
  Running from scipy source directory.
  lapack_opt_info:
  lapack_mkl_info:
  No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
  customize MSVCCompiler
    libraries mkl_rt not found in ['D:/Anaconda3\\Library\\lib']
    NOT AVAILABLE
  
  openblas_lapack_info:
    libraries openblas not found in 

   from D:\Anaconda3\Lib\site-packages\~cipy-1.5.2.dist-info
  Moving to d:\anaconda3\lib\site-packages\scipy\
   from D:\Anaconda3\Lib\site-packages\~cipy


      numpy/distutils/site.cfg file (section [lapack_src]) or by setting
      the LAPACK_SRC environment variable.
    return getattr(self, '_calc_info_{}'.format(name))()
    NOT AVAILABLE
  
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "C:\Users\Eric\AppData\Local\Temp\pip-install-bpwo_rzc\scipy\setup.py", line 492, in <module>
      setup_package()
    File "C:\Users\Eric\AppData\Local\Temp\pip-install-bpwo_rzc\scipy\setup.py", line 488, in setup_package
      setup(**metadata)
    File "D:\Anaconda3\lib\site-packages\numpy\distutils\core.py", line 135, in setup
      config = configuration()
    File "C:\Users\Eric\AppData\Local\Temp\pip-install-bpwo_rzc\scipy\setup.py", line 395, in configuration
      raise NotFoundError(msg)
  numpy.distutils.system_info.NotFoundError: No lapack/blas resources found.
  ----------------------------------------
  ERROR: Failed building wheel for scipy
  ERROR: Command errored out with exit status 1:
   co

In [28]:
import numpy as np
from PIL import Image
import scipy.misc as im

import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from skimage.transform import resize
from skimage import data

In [39]:
import PIL #resizing the images
import imageio

**Data Processing**

![alt text](https://drive.google.com/uc?export=view&id=11u7TILbn_1a5OvI97Zrv4nzGke8LSC88)

![alt text](https://drive.google.com/uc?export=view&id=1LtSjZGAiqjl6UPZYBJjiJupzPyyA6YGN)

In [79]:
def get_mstar_data(stage, width=100, height=100, crop_size=100, aug=False):
    data_dir = "C:/Users/Eric/Desktop/Target-Detection-in-MSTAR-Images-master/train/" if stage == "train" else "C:/Users/Eric/Desktop/Target-Detection-in-MSTAR-Images-master/test/" if stage == "test" else None
    print("------ " + stage + " ------")
    sub_dir = ["2S1", "SN_132", "BRDM_2", "BTR_60", "SN_9563", "D7", "T62", "SN_C71", "ZIL131", "ZSU_23_4"]
    X = []
    y = []

    for i in range(len(sub_dir)):
        tmp_dir = data_dir + sub_dir[i] + "/"
        img_idx = [x for x in os.listdir(tmp_dir) if (x.endswith(".jpg")or x.endswith(".JPG"))]
        print(sub_dir[i], len(img_idx))
        y += [i] * len(img_idx)
        for j in range(len(img_idx)):
#             img = im.imresize(im.imread((tmp_dir + img_idx[j])), [height, width])
            img = np.array(Image.fromarray(imageio.imread((tmp_dir + img_idx[j]))).resize((height, width)))
            img = img[(height - crop_size) // 2 : height - (height - crop_size) // 2, \
                  (width - crop_size) // 2: width - (width - crop_size) // 2]
#             print(img)

            # img = img[16:112, 16:112]   # crop
            X.append(img)

    return np.asarray(X), np.asarray(y)

In [51]:
x='a.JPG'
x.endswith(".jpg") or x.endswith(".JPG")

True

In [53]:
def data_shuffle(X, y, seed=0):
    data = np.hstack([X, y[:, np.newaxis]])
    np.random.shuffle(data)
    return data[:, :-1], data[:, -1]

In [114]:
def (y_train, y_test):
    one_hot_trans = OneHotEncoder().fit(y_train[:, np.newaxis])
    return one_hot_trans.transform(y_train[:, np.newaxis]).toarray(), one_hot_trans.transform(y_test[:, np.newaxis]).toarray()

In [94]:
# 减去行均值
def mean_wise(X):
    return (X.T - np.mean(X, axis=1)).T

In [113]:

X=np.random.randn(5,5)
print(X)
print(X.T)
print(np.mean(X, axis=1))

[[ 0.0770124  -0.33174871  0.35002803  1.33668989 -0.81301079]
 [ 1.15921851  1.02782537  0.57484233 -1.56171812 -0.24830554]
 [ 0.72192298 -0.07461234 -0.06171017  0.59693475 -0.89229471]
 [-0.9619392   0.28028597  2.69606356  1.91827587 -1.59256956]
 [-0.95785167 -0.70939964 -0.51972874  0.44336478 -0.99823236]]
[[ 0.0770124   1.15921851  0.72192298 -0.9619392  -0.95785167]
 [-0.33174871  1.02782537 -0.07461234  0.28028597 -0.70939964]
 [ 0.35002803  0.57484233 -0.06171017  2.69606356 -0.51972874]
 [ 1.33668989 -1.56171812  0.59693475  1.91827587  0.44336478]
 [-0.81301079 -0.24830554 -0.89229471 -1.59256956 -0.99823236]]
[ 0.12379416  0.19037251  0.0580481   0.46802333 -0.54836953]


**Principal Component Analysis for Optimized Processing**

In [7]:
def pca(X_train, X_test, n):
    pca_trans = PCA(n_components=n).fit(X_train)
    return pca_trans.transform(X_train), pca_trans.transform(X_test)

**Analyzing Various Machine Learning Algorithms**

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

**Decision Tree Classifier**

In [9]:
def dt(criterion="entropy", max_features="sqrt"):
    return DecisionTreeClassifier(criterion=criterion, max_features=max_features, max_depth=None, random_state=0)

**Random Forest Classification**

In [10]:
def rf(n_tree=100, max_features="sqrt"):
    return RandomForestClassifier(n_estimators=n_tree, max_features=max_features, min_samples_split=2, \
                                  max_depth=None, bootstrap=True, oob_score=False, random_state=0, n_jobs=4)

**Gradient Boosting Classification**

In [11]:
def gbdt(n_tree=100, max_features="sqrt"):
    return GradientBoostingClassifier(n_estimators=n_tree, learning_rate=0.005, \
                                      max_features=max_features, max_depth=None, random_state=0)

**Logistic Regression**

In [12]:
def logit(C=1.0):
    return LogisticRegression(C=1.0, solver="lbfgs", max_iter=1000, random_state=0)

**Multi-layer Perceptron Classifier**

In [13]:
def mlp(hidden=(100), act="logistic", batch=32):
    return MLPClassifier(hidden_layer_sizes=hidden, activation=act, solver="sgd", batch_size=batch, \
                         learning_rate="constant", learning_rate_init=0.1, early_stopping=False, max_iter=1000, random_state=0)

**Support Vector Machine Classifier**

![alt text](https://drive.google.com/uc?export=view&id=1ABtElBwMDn1dmSOenCGok6i5rhsaq4Ql)

In [14]:
def svm(C=1.0, kernel="rbf"):
    return SVC(C=C, kernel=kernel, max_iter=-1, random_state=0)

**K Nearest Neighbours Classification**

In [15]:
def knn(n_neighbors=10, weights="distance"):
    return KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, algorithm="auto")

**Naive Bayes Classification**

In [16]:
def bayes():
    return GaussianNB()

**Trainer Fuction**

In [17]:
def train(X, y, classifier):
    return classifier.fit(X, y)

**Testing Function**

In [18]:
def test(X, classifier):
    return classifier.predict(X)

**Accuracy Function**

In [19]:
def acc(X, y, classifier):
    return classifier.score(X, y)

**Implementation of Algorithms real time in order to find the most optimized solution**

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.cm as cm

*Let's load the processed M-STAR dataset*

In [123]:
print("loading ... ")
X_train, y_train = get_mstar_data("train", 100, 100, 72)
X_test, y_test = get_mstar_data("test", 100, 100, 72)
X_train = np.reshape(X_train, [X_train.shape[0], X_train.shape[1] * X_train.shape[2]])
X_test = np.reshape(X_test, [X_test.shape[0], X_test.shape[1] * X_test.shape[2]])
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

loading ... 
------ train ------
2S1 274
SN_132 232
BRDM_2 274
BTR_60 195
SN_9563 233
D7 274
T62 273
SN_C71 233
ZIL131 274
ZSU_23_4 274
------ test ------
2S1 299
SN_132 196
BRDM_2 298
BTR_60 256
SN_9563 195
D7 299
T62 299
SN_C71 196
ZIL131 299
ZSU_23_4 299
(2536, 5184) (2536,) (2636, 5184) (2636,)


*Shuffling the loaded dataset to eliminate any bias*

In [124]:
print("shuffling ... ")
X_train, y_train = data_shuffle(X_train, y_train)
X_test, y_test = data_shuffle(X_test, y_test)

shuffling ... 


*Preprocessing the dataset*

In [125]:
print("preprocessing ...")
X_train = X_train / 255.0
X_test = X_test / 255.0
X_train = mean_wise(X_train)
X_test = mean_wise(X_test)
print(X_train)
X_train, X_test = pca(X_train, X_test, 80)

# y_train, y_test = one_hot(y_train, y_test)

preprocessing ...
[[ 0.03944565  0.08650448  0.00415154 ... -0.16055435 -0.04290729
  -0.18016219]
 [ 0.0757005   0.05217108 -0.0497897  ... -0.13998578 -0.10861323
  -0.08900539]
 [ 0.09396711 -0.07073877 -0.1177976  ... -0.05113093 -0.19622897
  -0.10211132]
 ...
 [-0.21021393  0.09174685  0.24468803 ... -0.19844923  0.16233509
   0.19762921]
 [ 0.04450042 -0.01432311 -0.22608781 ... -0.02608781 -0.09667605
  -0.13197016]
 [ 0.04343379 -0.16048778  0.06304164 ...  0.14539458 -0.01538974
  -0.0820564 ]]


In [119]:
print(X_train)

[[ 4.37967693e-04 -1.53353619e-02 -1.05917483e-03 ... -9.97725139e-04
   1.64346221e-03  1.56938264e-17]
 [ 2.14723636e-02  1.75083076e-02  1.35477782e-03 ... -1.18781432e-04
  -3.78044225e-03 -2.39608680e-17]
 [-1.60864097e-02 -2.11139627e-03 -1.23877131e-02 ... -1.13849091e-03
   8.62520018e-04 -6.77626358e-20]
 ...
 [-1.34575246e-02 -2.36927495e-02  1.54040468e-02 ... -8.24669339e-04
   2.18930912e-05  2.48485585e-17]
 [-1.29196537e-02  1.75324883e-02  2.36264421e-03 ...  1.71924344e-04
   1.06671989e-03 -1.96782694e-17]
 [ 5.37434625e-03  1.57120214e-02  7.99840812e-03 ... -2.22214175e-03
   1.70611602e-03 -1.94614290e-17]]


*Finally Training the model*

In [126]:
print("training ...")
# classifier = train(X_train, y_train, dt("entropy", 0.8)) # 70.68%
# classifier = train(X_train, y_train, rf(1000, "sqrt")) # 96.49%
# classifier = train(X_train, y_train, gbdt(1000, "sqrt")) # 95.17%
# classifier = train(X_train, y_train, logit(1.0))    # 90.14%
# classifier = train(X_train, y_train, mlp(1000, "logistic"))   # 93.36%
classifier = train(X_train, y_train, svm(1.0, "rbf"))     # 96.82%
# classifier = train(X_train, y_train, knn(10, "uniform"))   # 95.34%
# classifier = train(X_train, y_train, svm())   # 97.81%

training ...


*As seen while analyzing the above algorithms, Support Vector machine was able to provide higest accuracy for determining the vehicle images from the test data, that is 97.8% accuracy.*

In [127]:
print("testing ...")
print(acc(X_train, y_train, classifier))
print(acc(X_test, y_test, classifier))

testing ...
0.9988170347003155
0.9810318664643399


**The END**

**Thank You**